In [ ]:
%pip install -U transformers

In [ ]:
%pip install srt

## Local Inference on GPU
Model page: https://huggingface.co/openai/whisper-large-v3-turbo

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/openai/whisper-large-v3-turbo)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-large-v3-turbo")

In [ ]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-large-v3-turbo")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-large-v3-turbo")

In [ ]:
import torch
from transformers import pipeline
import srt
from datetime import timedelta

# 1. Khởi tạo pipeline
# Lưu ý: 'device' nên được thiết lập rõ ràng nếu bạn dùng GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v3-turbo",
    chunk_length_s=30,
    device=device,
)

print("Đang xử lý video... Vui lòng đợi.")

# 2. Gọi pipeline với tham số quan trọng: return_timestamps=True
# batch_size=8 giúp tăng tốc độ xử lý trên GPU
result = pipe("Test.mp4", return_timestamps=True, batch_size=8)

# 3. Lấy dữ liệu chunks
segments = result.get("chunks")

if segments is None:
    print("Không tìm thấy segments nào. Có thể file âm thanh quá ngắn hoặc im lặng.")
    exit()

subs = []
i = 1

for seg in segments:
    # Kiểm tra an toàn: Đôi khi timestamp có thể bị None ở cuối file
    if seg["timestamp"] is None:
        continue

    start_sec = seg["timestamp"][0]
    end_sec = seg["timestamp"][1]

    # Xử lý trường hợp end_sec bị None (thường gặp ở đoạn cuối cùng)
    if end_sec is None:
        # Nếu không có thời gian kết thúc, ta cộng thêm 2 giây hoặc bỏ qua
        end_sec = start_sec + 2.0

    start = timedelta(seconds=start_sec)
    end = timedelta(seconds=end_sec)
    text = seg["text"].strip()

    # Chỉ thêm vào nếu có nội dung text
    if text:
        subs.append(srt.Subtitle(i, start, end, text))
        i += 1

# 4. Ghi file SRT
with open("output.srt", "w", encoding="utf-8") as f:
    f.write(srt.compose(subs))

print(f"Xong! Đã tạo file output.srt với {i-1} câu phụ đề.")

In [ ]:
print(open("output.srt", "r", encoding="utf-8").read())